https://blog.fastforwardlabs.com/2018/04/10/pytorch-for-recommenders-101.html

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import utils
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
device

device(type='cuda')

In [2]:
rating, user_column_index, item_column_index = utils.get_movielens_data()

(100000, 4)
(1682, 24)
(943, 5)


In [3]:
rating = rating[['userid', 'itemid', 'rating', 'timestep']]

rating = rating.groupby("userid").agg({
    "itemid": list,
    "rating": list,
    "timestep": "count"
}).reset_index().rename(columns = {"timestep": "count"})

In [4]:
rating

,userid,itemid,rating,count
0,1,"[61, 189, 33, 160, 20, 202, 171, 265, 155, 117...","[4, 3, 4, 4, 4, 5, 5, 4, 2, 3, 4, 4, 5, 5, 4, ...",271
1,2,"[292, 251, 50, 314, 297, 290, 312, 281, 13, 28...","[4, 5, 5, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 4, 5, ...",62
2,3,"[335, 245, 337, 343, 323, 331, 294, 332, 328, ...","[1, 1, 1, 3, 2, 4, 2, 1, 5, 3, 3, 1, 4, 2, 3, ...",54
3,4,"[264, 303, 361, 357, 260, 356, 294, 288, 50, 3...","[3, 5, 5, 4, 4, 3, 5, 4, 5, 5, 4, 5, 3, 5, 3, ...",24
4,5,"[2, 17, 439, 225, 110, 454, 424, 1, 363, 98, 1...","[3, 4, 1, 2, 1, 1, 1, 4, 3, 3, 3, 4, 5, 2, 4, ...",174
...,...,...,...,...
938,939,"[931, 106, 258, 1054, 689, 476, 409, 121, 1190...","[2, 3, 4, 4, 5, 5, 4, 5, 5, 4, 5, 4, 5, 3, 2, ...",49
939,940,"[193, 568, 14, 205, 272, 655, 315, 66, 873, 28...","[3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 5, 5, 4, 3, 5, ...",107
940,941,"[147, 124, 117, 181, 993, 258, 7, 475, 257, 15...","[4, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 2, 2, 3, 5, ...",22
941,942,"[117, 200, 604, 423, 261, 427, 487, 323, 615, ...","[4, 4, 4, 5, 4, 5, 4, 3, 3, 4, 5, 3, 3, 2, 4, ...",79


In [ ]:
rating["rating"] = rating["rating"]/5
# rating["rating"] =  MinMaxScaler().fit_transform(rating[["rating"]].values).reshape(-1)

In [ ]:
n_users = rating["userid"].max() + 1
n_items = rating["itemid"].max() + 1
print(n_users, n_items)
train_df, test_df = utils.movielens_train_test_split(rating)
train_df.shape, test_df.shape

train_df, val_df = utils.movielens_train_test_split(train_df)
train_df.shape, val_df.shape

In [ ]:
test_df["rating"].hist()

In [ ]:
print(np.mean(np.abs(test_df["rating"].values - test_df["rating"].mean())))
print(np.mean(np.abs(test_df["rating"].values - train_df["rating"].mean())))
print(np.mean(np.abs(test_df["rating"].values - train_df["rating"].mode()[0])))
print(np.mean(np.abs(test_df["rating"].values - test_df["rating"].mode()[0])))

# Model definition

In [ ]:
class DenseNetWithFeatures(nn.Module):
    def __init__(self, n_users, n_items, n_factors, H1, D_out, n_user_features,
                 n_item_features):
        """
        Simple Feedforward with Embeddings
        """
        super().__init__()
        # user and item embedding layers
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=False)
        # linear layers
        self.linear1 = torch.nn.Linear(n_factors * 4, H1)
        self.linear2 = torch.nn.Linear(H1, D_out)
        self.user_features = nn.Sequential(
            torch.nn.Linear(n_user_features, n_factors), torch.nn.ReLU(),
            torch.nn.Linear(n_factors, n_factors))
        self.item_features = nn.Sequential(
            torch.nn.Linear(n_item_features, n_factors), torch.nn.ReLU(),
            torch.nn.Linear(n_factors, n_factors),
            torch.nn.Sigmoid()
        )

    def forward(self, users, items, user_features, item_features):
        users_embedding = self.user_factors(users)
        items_embedding = self.item_factors(items)
        user_features_embedding = self.user_features(user_features)
        item_features_embedding = self.item_features(item_features)
        # concatenate user and item embeddings to form input
        x = torch.cat([
            users_embedding, items_embedding, user_features_embedding,
            item_features_embedding
        ], 1)
        h1_relu = F.relu(self.linear1(x))
        output_scores = self.linear2(h1_relu)
        return output_scores

    def predict(self, users, items):
        # return the score
        output_scores = self.forward(users, items)
        return output_scores



In [ ]:
model = DenseNetWithFeatures(n_users,
                             n_items,
                             n_factors=20,
                             H1=20,
                             D_out=1,
                             n_user_features=len(user_column_index),
                             n_item_features=len(item_column_index))
loss_fn = torch.nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
# scheduler = None
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)
# lmbda = lambda epoch: 0.65 ** epoch
# scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
trainloader = DataLoader(train_df.values, batch_size=128, shuffle=True)
valloader = DataLoader(val_df.values, batch_size=128)
testloader = DataLoader(test_df.values, batch_size=128)
model = model.to(device)
model

In [ ]:
def evaluate(loader):
    maes = []
    for batch in loader:
        user = batch[:, 0].type(torch.LongTensor).to(device)
        item = batch[:, 1].type(torch.LongTensor).to(device)
        user_features = batch[:, user_column_index].type(torch.FloatTensor).to(device)
        item_features = batch[:, item_column_index].type(torch.FloatTensor).to(device)

        score = batch[:, 2].type(torch.FloatTensor)
        prediction = model(user, item, user_features, item_features)
        mae = mean_absolute_error(score.cpu().detach().numpy(),  prediction.cpu().detach().numpy())
        maes.append(mae)
    return np.mean(maes)

In [ ]:
losses = []
val_scores = []
test_scores = []
for epoch in range(100):
    losses_epoch = []
    for batch in tqdm(trainloader):
        optimizer.zero_grad()
        user = batch[:, 0].type(torch.LongTensor).to(device)
        item = batch[:, 1].type(torch.LongTensor).to(device)
        user_features = batch[:, user_column_index].type(torch.FloatTensor).to(device)
        item_features = batch[:, item_column_index].type(torch.FloatTensor).to(device)
        score = batch[:, 2].view(-1, 1).type(torch.FloatTensor)
        prediction = model(user, item, user_features, item_features)
        loss = loss_fn(prediction, score.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        losses_epoch.append(loss.item())
    if scheduler is not None:
        scheduler.step()
    losses.append(np.mean(losses_epoch))
    val_scores.append(evaluate(valloader))
    test_scores.append(evaluate(testloader))
    print(losses[-1])
    print(f"{epoch}) VAL {val_scores[-1]}, TEST {test_scores[-1]}")

In [ ]:
plt.figure(figsize = (12, 5))
plt.subplot(131)
plt.title("Losses")
plt.plot(losses)
plt.subplot(132)
plt.title("Val MAE")
plt.plot(val_scores)

plt.subplot(133)
plt.title("Test MAE")
plt.plot(test_scores)
plt.tight_layout()

In [ ]:
plt.figure(figsize = (12, 5))
plt.subplot(131)
plt.title("Losses")
plt.plot(losses)
plt.subplot(132)
plt.title("Val MAE")
plt.plot(val_scores)

plt.subplot(133)
plt.title("Test MAE")
plt.plot(test_scores)
plt.tight_layout()